In [1]:
import pandas as pd
import re
sheet_url = "https://docs.google.com/spreadsheets/d/1H2Fzwjw7u7wZmf3vMwrz2uKLsWHpUkRu4cWfrkLyuvA/edit?gid=1310034314#gid=1310034314"

sheet_id_match = re.search(r"/spreadsheets/d/([a-zA-Z0-9-_]+)", sheet_url)
sheet_id = sheet_id_match.group(1)
print(sheet_id)

sheet_name = "CxL"

csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df = pd.read_csv(csv_url)

1H2Fzwjw7u7wZmf3vMwrz2uKLsWHpUkRu4cWfrkLyuvA


In [ ]:
from openai import OpenAI
import urllib.parse

OPENAI_API_KEY  = ""

column_names = ", ".join([col for col in df.columns if not col.startswith("Unnamed")])
#table_name = "your_table"

#print(column_names)

user_prompt = "group by last name" #if columns name in query is col1, col2 they must be changed to A,B,C etc
#user_prompt = "show records where last name is yadav or YADAV"  #for where clause we have to take care of case sensitivity
#user_prompt = "sort by last name"
#user_prompt = "show all records"

# Create a schema description
schema_description = f"The table '{sheet_name}' has the following columns: {column_names}."

#print(schema_description)

# OpenAI LLM prompt
prompt = f"""
You are a Google Sheets formula expert, specializing in optimizing and structuring data using the QUERY() function.

Given the following table schema:

{schema_description}

Generate an optimized Google Sheets QUERY() formula to fulfill the following request:
"{user_prompt}"

### Requirements:
- Ensure the QUERY() function is correctly structured with valid Google Sheets syntax.
- Use appropriate column references and data ranges.
- Format the query efficiently to handle large datasets if applicable.
- Apply sorting, filtering, grouping, or aggregations where relevant.
- Ensure the formula dynamically adjusts for new data when feasible.
- While writing the query consider the case sensitivity of the data.
- While writing the query mention column names as A,B,C etc. instead of actual column names.

If the QUERY() function alone cannot achieve the request, suggest alternative Google Sheets formulas or functions (e.g., FILTER(), ARRAYFORMULA(), REGEXMATCH()) that complement it.
"""
openai_client = OpenAI(api_key=OPENAI_API_KEY)
# OpenAI API call
response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a Google Sheets formula expert, specializing in optimizing and structuring data using the QUERY() function"},
        {"role": "user", "content": prompt}],
    
)

query = response.choices[0].message.content

#print(query)

match = re.search(r'QUERY\([^,]+,\s*"([^"]+)"', query, re.IGNORECASE)
q1 =  match.group(1) if match else None
q2 = urllib.parse.quote(q1)

#fetch_query = "SELECT B, COUNT(B) GROUP BY B LABEL COUNT(B) 'Total Count'"
#fetch_query_encoded = urllib.parse.quote(fetch_query)

#print("QUERY" , fetch_query_encoded)



result_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&tq={q2}&sheet={sheet_name}"

# Fetch Data
result_df = pd.read_csv(result_url)

# Display results
print(result_df)

#print(response.choices[0].message.content)

      Last Name  Total Count
0           NaN            0
1             1            1
2             2            1
3             3            1
4             4            1
..          ...          ...
118  Y Mudlawar            1
119       YADAV            1
120       Yadav            3
121   Yammanaru            1
122       kumar            1

[123 rows x 2 columns]
